# Bayesian Model

This notebook illustrate how to use a Bayesian Normal density model with the [beer framework](https://github.com/beer-asr/beer). The Normal distribution is a fairly basic models but it is used extenslively in other model as a basic building block.

In [ ]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.insert(0, '../')

import beer
import numpy as np
import torch

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

# Convenience functions for plotting.
import plotting

%load_ext autoreload
%autoreload 2

## Data

Generate some normally distributed data:

In [ ]:
mean = np.zeros(2) 
cov = np.array([
    [3, 1.9],
    [1.9, 2]
])
data = np.random.multivariate_normal(mean, cov, size=100)

fig = figure(
    title='Data',
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, mean, cov, line_color='black', fill_alpha=.3)

show(fig)

## Model Creation

We create two types of Normal distribution: one diagonal covariance matrix and another one with full covariance matrix.

In [ ]:
data_mean = torch.from_numpy(data.mean(axis=0)).float()
data_var = torch.from_numpy(data.var(axis=0)).float()


normal_iso = beer.create_model(
    {
        'type': 'Normal',
        'covariance': 'isotropic',
        'prior_strength': 1.,
        'noise_std': 0.
    },
    data_mean, data_var
)

normal_diag = beer.create_model(
    {
        'type': 'Normal',
        'covariance': 'diagonal',
        'prior_strength': 1.,
        'noise_std': 0.
    },
    data_mean, data_var
)

normal_full = beer.create_model(
    {
        'type': 'Normal',
        'covariance': 'full',
        'prior_strength': 1.,
        'noise_std': 0.
    },
    data_mean, data_var
)

models = {
    'normal_full': normal_full,
    'normal_diag': normal_diag,
    'normal_iso': normal_iso
}

## Variational Bayes Training 

In [ ]:
epochs = 3
lrate = 1.
X = torch.from_numpy(data).float()

optims = {
    model_name: beer.BayesianModelCoordinateAscentOptimizer(
        model.mean_field_groups, lrate)
    for model_name, model in models.items()
}

elbos = {
    model_name: [] 
    for model_name in models
}  


for epoch in range(epochs):
    for name, model in models.items():
        optim = optims[name]
        optim.zero_grad()
        elbo = beer.evidence_lower_bound(model, X, datasize=len(X))
        elbo.natural_backward()
        elbos[name].append(float(elbo) / len(X))
        optim.step()

In [ ]:
colors = {
    'normal_iso': 'green',
    'normal_diag': 'blue',
    'normal_full': 'red',
    
}
# Plot the ELBO.
fig = figure(title='ELBO', width=400, height=400, x_axis_label='step',
              y_axis_label='ln p(X)')
for model_name, elbo in elbos.items():
    fig.line(range(len(elbo)), elbo, legend=model_name, color=colors[model_name])
fig.legend.location = 'bottom_right'

show(fig)

In [ ]:
fig = figure(
    width=400,
    height=400,
    x_range=(mean[0] - 5, mean[0] + 5),
    y_range=(mean[1] - 5, mean[1] + 5)
)
fig.circle(data[:, 0], data[:, 1])
plotting.plot_normal(fig, normal_iso.mean.numpy(), normal_iso.cov.numpy(), 
                     line_color='black', alpha=.5, color='green')
plotting.plot_normal(fig, normal_diag.mean.numpy(), normal_diag.cov.numpy(), 
                     line_color='black', alpha=.5, color='#98AFC7')
plotting.plot_normal(fig, normal_full.mean.numpy(), normal_full.cov.numpy(), 
                     line_color='black', alpha=.5, color='#C7B097')

show(fig)